# Easy Tensorflow:

An interface containing easy tensorflow model building blocks and feature encoding pipelines. This module contains functionality similar to what sklearn does with its Pipeline, FeatureUnion and ColumnTransformer. 

Pipeline types:
- Encoder: Preprocess feature based on specified preprocessing layer 
- SequentialEncoder: Preprocessing pipeline to apply multiple encoders in serie
- Pipeline:  Main interface for transforming features. Apply feature encoder list which can contain both Encoder and SequentialEncoder object types
- FeatureUnion: Similar tp Pipeline, but concats the Layers from Pipeline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%pip install -q easy-tensorflow==1.1.2

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Normalization, IntegerLookup, StringLookup

# import from easyflow
from easyflow.data.mapper import TensorflowDataMapper
from easyflow.preprocessing import Pipeline, FeatureUnion

In [ ]:
dataframe = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
labels = dataframe.pop("target")

batch_size = 32
dataset_mapper = TensorflowDataMapper() 
dataset = dataset_mapper.map(dataframe, labels)
train_data_set, val_data_set = dataset_mapper.split_data_set(dataset)
train_data_set = train_data_set.batch(batch_size)
val_data_set = val_data_set.batch(batch_size)

NUMERICAL_FEATURES = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']
CATEGORICAL_FEATURES = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca', 'thal']

## Setup Preprocessing layer using FeatureUnion

In [ ]:
feature_encoder_list = [
                        ('numeric_encoder', Normalization(), NUMERICAL_FEATURES),
                        ('categorical_encoder', IntegerLookup(output_mode='binary'), CATEGORICAL_FEATURES)
                        ]

encoder = FeatureUnion(feature_encoder_list)
all_feature_inputs, preprocessing_layer = encoder.encode(train_data_set)

In [ ]:
# setup simple network
x = tf.keras.layers.Dense(128, activation="relu")(preprocessing_layer)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=all_feature_inputs, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.AUC(name='auc')])

history=model.fit(train_data_set, validation_data=val_data_set, epochs=10)

In [ ]:
model.predict(val_data_set)[:10]